## Clustering Methods in Python
In this vignette, we will be comparing three different clusterig methods in python. We will be demonstrating this using a data set that deals with user behavior depending on what kind of phone they have.

### Objective:

### Set up:

##### Importing modules:

##### Partitioning data:

## Clustering Methods:
We will be comparing three different clustering methods today: k means, spectral, and agglomerative.

#### K-Means Clustering:
K-means clustering is an unsupervised machine learning algorithm which groups dataset into various clusters. It works by separating samples into groups of equal variances. This algorithim will follow 3 main steps:
- Initialize k points, called means or cluster centroids.
- Assign data points to clusters.
- Update the coordinates of the centroids and repeat the process.

#### Spectral Clustering:
Spectral clustering is also an unsupervised machine learning algorithim. It is based on spectral graph theory and linear algebra. The main idea is to separate the data into several smaller groups. It is used mainly for clustering non-linearly spearable data. This algorithim will follow 2 main steps:
- Creating a similarity matrix based on the data
- Finding the eigenvectors and using them to transform the data into a lower-dimensional space. 

#### Agglomerative Clustering:
Lastly, we have agglomerative clustering which is also an unsupervised machine learning algorithim. It is known as the "bottom-up" approach. Each data point is initially labeled as a cluster. It then works recursively to merge and form larger clusters. There are 3 main steps:
- Creating a proximity matrix
- Using a linkage function to group objects into a hierarchical cluster tree
- Merging clusters togther that are in close proximity